In [1]:
!pip install selenium

DEPRECATION: Loading egg at c:\users\kriti\appdata\local\programs\python\python311\lib\site-packages\face_recognition-1.3.0-py3.11.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330


In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import csv
import tempfile

In [3]:
# Setup Selenium WebDriver
options = webdriver.ChromeOptions()
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("--start-maximized")

driver = webdriver.Chrome(options=options)

In [4]:
# Define search query
search_query = "restaurants in Downtown Toronto"
google_url = f"https://www.google.com/search?q={search_query.replace(' ', '+')}"
driver.get(google_url)
time.sleep(3)  # Allow page to load

In [5]:
# Scroll down to load more results
for _ in range(5):  
    driver.find_element(By.TAG_NAME, "body").send_keys(Keys.PAGE_DOWN)
    time.sleep(3)  # for new results to load

In [6]:
# Find restaurant listings
restaurants = []
results = driver.find_elements(By.CLASS_NAME, "VkpGBb")  
print("Found", len(results), "results.")

Found 3 results.


In [7]:
for i, result in enumerate(results):
    try:
        name = result.find_element(By.CLASS_NAME, "dbg0pd").text  # Restaurant name
    except Exception:
        name = "N/A"
    
    try:
        rating_element = result.find_element(By.XPATH, ".//span[contains(@aria-label, 'Rated')]")
        rating = rating_element.get_attribute("aria-label")
    except Exception:
        rating = "N/A"
    
    try:
        address = result.find_element(By.CLASS_NAME, "rllt__details").text  # Address
    except Exception:
        address = "N/A"
    
    try:
        phone_number = result.find_element(By.CLASS_NAME, "rllt__wrapped").text  # Phone number (if available)
    except Exception:
        phone_number = "N/A"
    
    print(f"Result {i}: {name}, {rating}, {address}, {phone_number}")
    restaurants.append([name, rating, address, phone_number])  

Result 0: Richmond Station, Rated 4.7 out of 5,, Richmond Station
4.7(3.3K) · $100+ · Canadian
1 Richmond St W
Closed ⋅ Opens 11:30 am
Chef-driven stop for seasonal dishes, $100+
Result 1: Byblos Downtown, Rated 4.5 out of 5,, Byblos Downtown
4.5(3.4K) · ₹₹₹ · Middle Eastern
11 Duncan St
Closed ⋅ Opens 5 pm
Upscale Eastern Mediterranean dining, N/A
Result 2: JOEY Eaton Centre, Rated 4.5 out of 5,, JOEY Eaton Centre
4.5(6.5K) · ₹₹ · Restaurant
1 Dundas St W
Closes soon ⋅ 1 am
Global comfort food in a modern setting, N/A


In [8]:
import pandas as pd

df = pd.DataFrame(restaurants, columns=["Name", "Rating", "Address", "Phone Number"])

# Save to CSV
csv_file = "restaurants_selenium.csv"
df.to_csv(csv_file, index=False, encoding="utf-8-sig")
print(f"Data saved to {csv_file}")

Data saved to restaurants_selenium.csv


In [ ]:
'''
NOTE: Getting all the details through selenium is a bit risky as we are using google url for search, I tried to get the information about "more places" 
but it was beyond their security purpose.

Instead we can use "WebBaseLoader" and a LLM to get  